<a href="https://colab.research.google.com/github/Bryant1217/Predict-Diamond-price/blob/main/Diamond_(drop_variables).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px

In [4]:
df = pd.read_excel("UV6248-XLS-ENG.xls", sheet_name = "Raw Data")
df = df.rename(columns={'Carat Weight': 'CaratWeight'})
df = df.dropna()
df1= df.copy()

Check the number of observations across categories in all variables.

For problematic categories, combine the ''rare categories" together.

Try this on your own with your teammates. If you do not how, ask ChatGPT.

In [16]:
df1.Clarity.unique()
print(df1.Clarity.unique())
FL_count2 = df1['Clarity'].value_counts()
print(FL_count2)

# FL is quite low and consider combine or drop it
df1['Clarity']= df1['Clarity'].replace({'IF': 'IF_FL','FL': 'IF_FL'})
df1.Clarity.unique()
print(df1.Clarity.unique())

[1 5 4 2 3 'IF_FL']
1        2059
4        1575
5        1192
2         666
3         285
IF_FL     223
Name: Clarity, dtype: int64
[1 5 4 2 3 'IF_FL']


Step 2.2 Visual Exploration of the Data
Explore impact of different categorical variables on our data.

In [7]:
fig = px.scatter(df1,
                 x="CaratWeight",
                 y="Price",
                 color = 'Color',
                 height = 350
                )
fig.show()


In [8]:
#explore whether taking a log of X or Y helps the relationship.
from math import log
#by doing copy the original data won't be change by all the number being log
df_log = df.copy()
df_log['Price']=df_log['Price'].transform(log)
#df_log['CaratWeight']=df_log['CaratWeight'].transform(log)
print(df_log['Price'].head(10))

fig = px.scatter(df_log,
                 x="CaratWeight",
                 y="Price",
                 color = 'Color',
                 height = 350,
                 labels={'Price':'Log of Price'#, 'CaratWeight': 'Log of CaratWeight'
                         }
                )
fig.show()

0    8.550435
1    8.151910
2    8.065579
3    8.382518
4    8.061802
5    9.456497
6    8.656433
7    9.254357
8    9.831401
9    8.944550
Name: Price, dtype: float64


Create dummy variables for categorical variables using pd.get_dummies():

In [9]:
df1 = pd.get_dummies(data=df, columns = ['Color', 'Report'], drop_first=True)
df1.head() # Check how the data looks now
# ['Color','Clarity','Cut','Polish','Symmetry','Report']

,ID,CaratWeight,Cut,Clarity,Polish,Symmetry,Price,Color_E,Color_F,Color_G,Color_H,Color_I,Report_GIA
0,1,1.10,Ideal,SI1,VG,EX,5169.0,0,0,0,1,0,1
1,2,0.83,Ideal,VS1,ID,ID,3470.0,0,0,0,1,0,0
2,3,0.85,Ideal,SI1,EX,EX,3183.0,0,0,0,1,0,1
3,4,0.91,Ideal,SI1,VG,VG,4370.0,1,0,0,0,0,1
4,5,0.83,Ideal,SI1,EX,EX,3171.0,0,0,1,0,0,1


Look at the correlation matrix to see if we have potential multicollinearity

Encode ordinal categorical variables using an increasing scale

In [23]:
scale_map_cut = {'Fair':1, 'Good':2, 'Very Good':3, 'Ideal':4,  'Signature-Ideal':5}
scale_map_clarity = {'SI1':1, 'VVS2':2, 'VVS1':3, 'VS2':4, 'VS1':5, 'IF_FL':6}
scale_map_polish = {'G':1, 'VG':2, 'EX':3, 'ID':3}

df1["Cut"] = df1["Cut"].replace(scale_map_cut)
df1["Clarity"] = df1["Clarity"].replace(scale_map_clarity)
df1["Polish"] = df1["Polish"].replace(scale_map_polish)
df1["Symmetry"] = df1["Symmetry"].replace(scale_map_polish)
df1.tail()

,ID,CaratWeight,Cut,Clarity,Polish,Symmetry,Price,Color_E,Color_F,Color_G,Color_H,Color_I,Report_GIA
5995,5996,1.03,4,1,3,3,6250.0,0,0,0,0,0,1
5996,5997,1.00,3,1,2,2,5328.0,0,0,0,0,0,1
5997,5998,1.02,4,1,3,3,6157.0,0,0,0,0,0,1
5998,5999,1.27,5,5,3,3,11206.0,0,0,1,0,0,1
5999,6000,2.19,4,5,3,3,30507.0,1,0,0,0,0,1


In [25]:
print(df1.dtypes)
df1.corr().style.background_gradient(cmap='RdBu_r', axis=None)
df1.drop(['Price','ID'], axis=1).corr().style.background_gradient(cmap='coolwarm', axis=None)
#"axis=None" option above indicates that the colors are assigned based on the values in the whole matrix
# Other good color maps: 'RdBu_r' & 'PuOr_r' & 'coolwarm'

ID               int64
CaratWeight    float64
Cut              int64
Clarity          int64
Polish           int64
Symmetry         int64
Price          float64
Color_E          uint8
Color_F          uint8
Color_G          uint8
Color_H          uint8
Color_I          uint8
Report_GIA       uint8
dtype: object


,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I,Report_GIA
CaratWeight,1.000000,0.072943,0.147170,0.068080,0.054716,-0.089155,-0.024898,0.037891,0.045767,0.058235,0.011461
Cut,0.072943,1.000000,0.106612,0.398449,0.520504,-0.053341,-0.045228,0.057314,0.004432,0.012034,-0.276354
Clarity,0.147170,0.106612,1.000000,0.073543,0.057909,-0.074843,-0.009883,0.111900,-0.021487,0.003519,-0.044425
Polish,0.068080,0.398449,0.073543,1.000000,0.593765,-0.033634,-0.026723,0.062853,0.005671,-0.006967,-0.285906
Symmetry,0.054716,0.520504,0.057909,0.593765,1.000000,-0.059661,-0.017800,0.054295,0.013411,0.006019,-0.304807
Color_E,-0.089155,-0.053341,-0.074843,-0.033634,-0.059661,1.000000,-0.173963,-0.222948,-0.180740,-0.169293,0.059315
Color_F,-0.024898,-0.045228,-0.009883,-0.026723,-0.017800,-0.173963,1.000000,-0.260326,-0.211042,-0.197675,0.028410
Color_G,0.037891,0.057314,0.111900,0.062853,0.054295,-0.222948,-0.260326,1.000000,-0.270468,-0.253338,-0.021583
Color_H,0.045767,0.004432,-0.021487,0.005671,0.013411,-0.180740,-0.211042,-0.270468,1.000000,-0.205377,-0.029140
Color_I,0.058235,0.012034,0.003519,-0.006967,0.006019,-0.169293,-0.197675,-0.253338,-0.205377,1.000000,-0.072709


Compute Variance Inflation Factors (VIF)

In [26]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe

vif_data1 = pd.DataFrame()
vif_data1["feature"] = df1.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data1["VIF"] = [variance_inflation_factor(df1.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df1.drop(['Price','ID'], axis=1).columns))]
print(vif_data1)

        feature        VIF
0   CaratWeight   8.661387
1           Cut  21.209733
2       Clarity   4.198909
3        Polish  21.106344
4      Symmetry  20.531366
5       Color_E   1.952773
6       Color_F   2.265679
7       Color_G   3.004288
8       Color_H   2.379213
9       Color_I   2.228594
10   Report_GIA   6.014738


Try mean-centering variables
Try creating combinations of variables
Try dropping variables

In [27]:
#drop 'Symmetry','Polish', 'Cut'
vif_data2 = pd.DataFrame()
vif_data2["feature"] = df1.drop(['Price','ID','Symmetry','Polish'], axis=1).columns


# calculating VIF for each feature
vif_data2["VIF"] = [variance_inflation_factor(df1.drop(['Price','ID','Symmetry','Polish'], axis=1).values, i)
                          for i in range(len(df1.drop(['Price','ID', 'Symmetry','Polish'], axis=1).columns))]
print(vif_data2)

       feature        VIF
0  CaratWeight   8.426367
1          Cut  10.244471
2      Clarity   4.174843
3      Color_E   1.907157
4      Color_F   2.208260
5      Color_G   2.917201
6      Color_H   2.321840
7      Color_I   2.185718
8   Report_GIA   5.974793


Split the data into X and Y.
The vector of Y ("dependent") variable should contain the Price. The matrix of X ("independent") variables should contain everything we will use to predict Y

In [28]:
Y = df1[(['Price'])]
Y.head() # it's always a good idea to peak at your output

,Price
0,5169.0
1,3470.0
2,3183.0
3,4370.0
4,3171.0


In [29]:
X = df1.drop(['Price','ID', 'Report_GIA','Symmetry','Polish'], axis=1)
#X = df[['CaratWeight']]
X.dtypes

CaratWeight    float64
Cut              int64
Clarity          int64
Color_E          uint8
Color_F          uint8
Color_G          uint8
Color_H          uint8
Color_I          uint8
dtype: object

Build regression models
3.1 Try a linear model

In [30]:
# In this package, by default, the regression will have no intercept, hence we need to manually add it to the X matrix, and call the result X_const
X_const = sm.add_constant(X)
X_const.head()
# Fit a linear regression model with vector Y as dependent and matrix X_sm as independent
lm = sm.OLS(Y, X_const).fit()

# Display the summary of model results
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     3461.
Date:                Mon, 22 Jan 2024   Prob (F-statistic):               0.00
Time:                        01:14:57   Log-Likelihood:                -58705.
No. Observations:                6000   AIC:                         1.174e+05
Df Residuals:                    5991   BIC:                         1.175e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.272e+04    317.419    -40.087      

Plot the residuals

In [31]:
# Compute the residuals
results = pd.DataFrame()
results['Price'] = df['Price']
results['prediction_lm'] = lm.fittedvalues
results['residual_lm'] = lm.resid

fig = px.scatter(
    results, x='prediction_lm', y='residual_lm', height = 350,
    labels={'prediction_lm':'Predicted values using the Linear Model', 'residual_lm':'Residuals using the Linear Model'}
)
fig.show()

log-linear model to the case data

In [32]:
from math import exp, log
Y_log = Y['Price'].transform(log)
log_linear_model = sm.OLS(Y_log, X_const).fit()
print(log_linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     9352.
Date:                Mon, 22 Jan 2024   Prob (F-statistic):               0.00
Time:                        01:15:02   Log-Likelihood:                 1343.9
No. Observations:                6000   AIC:                            -2670.
Df Residuals:                    5991   BIC:                            -2610.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.0658      0.014    494.320      

In [33]:
# Compute residuals
results['prediction_llm'] = log_linear_model.fittedvalues
results['residual_llm'] = log_linear_model.resid

fig = px.scatter(
    results, x='prediction_llm', y='residual_llm', height = 350,
    labels={'prediction_llm':'Predicted values using the Log-Linear Model', 'residual_llm':'Residuals'}
)
fig.show()

Fit a log-log model to the case data

In [34]:
from math import exp, log
Y_log = Y['Price'].transform(log)
#duplicate x for making the original data clean
X_log= X.copy()
X_log['CaratWeight'] = X_log['CaratWeight'].transform(log)
X_log_const= sm.add_constant(X_log)
X_log_const.head()
log_log_model = sm.OLS(Y_log, X_log_const).fit()
print(log_log_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.949
Method:                 Least Squares   F-statistic:                 1.388e+04
Date:                Mon, 22 Jan 2024   Prob (F-statistic):               0.00
Time:                        01:15:08   Log-Likelihood:                 2455.5
No. Observations:                6000   AIC:                            -4893.
Df Residuals:                    5991   BIC:                            -4833.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.4680      0.011    776.329      

In [35]:
# Compute residuals
results['prediction_llm'] = log_log_model.fittedvalues
results['residual_llm'] = log_log_model.resid

fig = px.scatter(
    results, x='prediction_llm', y='residual_llm', height = 350,
    labels={'prediction_llm':'Predicted values using the Log-Log Model', 'residual_llm':'Residuals'}
)
fig.show()

Cross-validation
The main machine learning principle that allows to answer this question -- cross-validation: splitting the data into training (80%) and testing (20%) subsets, training on the former and testing on the latter. Find the MAPE

In [36]:
# Redefine X and Y for the training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Add a constant to the X's:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

X_train_const.head()

,const,CaratWeight,Cut,Clarity,Color_E,Color_F,Color_G,Color_H,Color_I
3897,1.0,1.34,3,4,0,0,0,1,0
5628,1.0,2.57,4,4,0,0,0,1,0
1756,1.0,1.01,4,6,0,0,0,0,0
2346,1.0,2.09,4,5,0,1,0,0,0
2996,1.0,0.90,3,1,0,0,0,0,1


In [37]:
# Fit a linear regression model to the training data
lm = sm.OLS(Y_train, X_train_const).fit()

# Use the trained model to predict the prices for the testing data. Call the vector of predicted prices Y_pred
Y_pred = lm.predict(X_test_const)
percent_errors = np.abs((Y_test['Price'] - Y_pred) / Y_test['Price']) *100
print("Linear Model MAPE = ", np.mean(percent_errors), "%")

Linear Model MAPE =  29.407542785406367 %


In [38]:
# Fit a log-linear regression model to the training data       #nearly 2 time better than linear model
Y_train_log = Y_train['Price'].transform(log)
llm = sm.OLS(Y_train_log, X_train_const).fit()

Y_pred_llm = np.exp(llm.predict(X_test_const))
percent_errors = np.abs((Y_test['Price'] - Y_pred_llm) / Y_test['Price']) *100
print("Log-Linear Model MAPE = ", np.mean(percent_errors), "%")

Log-Linear Model MAPE =  14.367150805083371 %


In [39]:
# Fit a log-log regression model to the training data
X_train_log = X_train.copy()
X_train_log['CaratWeight'] = X_train_log['CaratWeight'].transform(log)
X_train_log_const = sm.add_constant(X_train_log)

X_test_log = X_test.copy()
X_test_log['CaratWeight'] = X_test_log['CaratWeight'].transform(log)
X_test_log_const = sm.add_constant(X_test_log)


loglog = sm.OLS(Y_train_log, X_train_log_const).fit()

Y_pred_loglog = np.exp(loglog.predict(X_test_log_const))
percent_errors = np.abs((Y_test['Price'] - Y_pred_loglog) / Y_test['Price']) *100
print("Log-Log Model MAPE = ", np.mean(percent_errors), "%")

Log-Log Model MAPE =  12.1640221409409 %
